<a href="https://colab.research.google.com/github/xlnt415/visualization/blob/main/dash/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9D%84%EC%9D%B4%EC%9A%A9%ED%95%9C%EC%9D%B8%ED%84%B0%EB%9E%99%ED%8B%B0%EB%B8%8C%EB%8C%80%EC%8B%9C%EB%B3%B4%EB%93%9C%EB%A7%8C%EB%93%A4%EA%B8%B0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DASH 실습

## 0. 라이브러리 호출

In [ ]:
# 라이브러리 호출

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 1. 기초

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/xlnt_space/dash/rawdata/Data.csv')

In [ ]:
# 연도, 월 변수 생성

df['year'] = df['OrderDate'].str.slice(start = 0, stop = 4)
df['month'] = df['OrderDate'].str.slice(start = 5, stop = 7)

# 데이터 정렬
df = df.sort_values(by = ['Region', 'Channel', 'Category', 'Item Type', 'year', 'month', 'Gender'])

In [ ]:
# 2020년도 이익 변수 생성
d20 = df[df['year'] == '2020'].copy()
d20['Margin'] = d20['Revenue'] - d20['Cost']

In [ ]:
# Country별 매출 및 이익 합계 산출
df_g = d20.loc[:, ['Country', 'Revenue', 'Margin']].groupby(by = ['Country'], as_index = False).sum()
df_g = df_g.sort_values(by = ['Revenue'], ascending = False)

In [ ]:
# 매출 순위 변수 [rank] 생성 후, 매출 상위 10개 Country 추출
df_g['rank'] = list(range(1, len(df_g['Country']) + 1))
df_g1 = df_g[df_g['rank'] <= 10].reset_index(drop = True)

df_g1



## 그래프

### 바그래프

In [ ]:
# Bar

trace = go.Bar(
    x = df_g1.Country,
    y = df_g1.Revenue,
    text = round(df_g1.Revenue, 2))

data = [trace]
layout = go.Layout(title = 'Chapter 2.1 - Bar Chart')

fig = go.Figure(data, layout)
fig.show()

### 중첩 바그래프

In [ ]:
# 중첩 BAR

trace1 = go.Bar(
    y = df_g1['Country'],
    x = df_g1.Revenue,
    name = 'Revunues',
    orientation = 'h'
)

trace2 = go.Bar(
    y = df_g1['Country'],
    x = df_g1.Margin,
    name = 'Margin',
    orientation = 'h'
)

data = [trace1, trace2]
layout = go.Layout(title = 'Chapter 2.1 - Bar Chart',
                   barmode = 'group',
                   yaxis = dict(autorange = 'reversed'))

fig = go.Figure(data, layout)
fig.show();

### Scatter & Line Chart

In [ ]:
df_g = df.loc[:, ['Revenue', 'year', 'month']].groupby(by = ['year', 'month'], as_index = False).sum()

year = list(df_g.year.unique())

In [ ]:
traces = []

for years in year:
    tmp = df_g[df_g.year == years]
    traces.append(go.Scatter(x = tmp.month,
                             y = tmp.Revenue,
                             mode = 'lines+markers',
                             marker = dict(size = 10),
                             name = years))


data = traces

layout = go.Layout(
    title = 'Chater2.2 - Scatter & Line Charts',
    xaxis = dict(title = 'Month'),
    yaxis = dict(title = 'Revenue')
)

fig = go.Figure(data, layout)

fig.show()

### Pie Chart

In [ ]:
df_g = df[df.year == '2020'].copy()

df_g1 = df_g.loc[:, ['AgeGroup', 'Revenue']].groupby(by = ['AgeGroup'], as_index = False).sum()

In [ ]:
trace = go.Pie(
    labels = df_g1.AgeGroup,
    values = df_g1.Revenue
)

data = [trace]

layout = go.Layout(title = 'Chapter 2.3 - Pie Chart')
fig = go.Figure(data, layout)

fig.show()

#### 조각 분리

In [ ]:
trace = go.Pie(
    labels = df_g1.AgeGroup,
    values = df_g1.Revenue,
    pull = [0, 0, 0.2, 0, 0] # label 순서와 동일(0~1범위), 코드 의미 : 40대 조각을 0.2만큼 분리
)

data = [trace]

layout = go.Layout(title = 'Chapter 2.3 - Pie Chart Split')

fig = go.Figure(data, layout)
fig.show()

### 도넛모양

In [ ]:
trace = go.Pie(
    labels = df_g1.AgeGroup,
    values = df_g1.Revenue,
    textinfo = 'label+percent', # text 값 형식
    insidetextorientation='tangential',   # testinfo 타입
    hole = 0.4  # 원 중심부 구멍 크기
)

data = [trace]

layout = go.Layout(title = 'Chapter 2.3 - Pie Chart Split')

fig = go.Figure(data, layout)
fig.show()

### box plot

In [ ]:
df_g = df[df.year == '2020'].loc[:, ['Region', 'Revenue']].copy()

regions = list(df_g.Region.unique())
regions.sort()

In [ ]:
trace = []

for region in regions:
    tmp = df_g[df_g.Region == region]
    trace.append(go.Box(
        y = tmp.Revenue,
        name = region
    ))

data = trace

layout = go.Layout(
    title = 'Chapter 3.1 - Box Plot'
)

fig = go.Figure(data, layout)

fig.show()

### Histogram

In [ ]:
df_g= df[df.year == '2020'].loc[:, ['AgeGroup', 'Quantity']].copy()

ages = list(df_g.AgeGroup.unique())

ages.sort()

|Options|내용|
|----|-----|
|True|동일한 행 내의 처음 그려진 그래프의 Y축 값을 공유|
|'columns'|동일한 열 내의 처음 그려진 그래프의 Y축 값을 공유|
|'all'|모든 그래프가 처음 그려진 그래프 Y축값을 공유|

In [ ]:
df_g.columns

In [ ]:
fig = make_subplots(rows = 2, cols = 3, shared_yaxes = 'all')

traces = []

for age in ages:
    trace.append(go.Histogram(x = df_g[df_g.AgeGroup == age]['Quantity'], name = age))

fig.append_trace(trace[0], 1, 1)
fig.append_trace(trace[1], 1, 2)
fig.append_trace(trace[2], 1, 3)
fig.append_trace(trace[3], 2, 1)
fig.append_trace(trace[4], 2, 2)

fig.update_layout(title = 'Chapter 3.2 - Histogram')

fig.show()

### Error Bar

- 표준편차 나태날 때 주로 사용

In [ ]:
df1 = df[(df.Region == 'Asia') & (df.Category == 'Foods')].copy()

In [ ]:
df_g = df1.loc[:, ['Channel', 'year', 'Revenue']].copy()

g_mean = df_g.groupby(['Channel', 'year'], as_index = False).mean()
g_std = df_g.groupby(['Channel', 'year'], as_index = False).std()
g_n = df_g.groupby(['Channel', 'year'], as_index = False).count()
df_g1 = pd.concat([g_mean.reset_index(drop = True),
                   g_std['Revenue'].reset_index(drop = True),
                   g_n.Revenue.reset_index(drop = True)],
                  axis = 1)

df_g1.columns = ['Channel', 'year', 'mean', 'sd', 'n']

In [ ]:
df_g2 = df_g1[df_g1['Channel'] == 'Offline'].copy()

trace = go.Scatter(x = df_g2.year,
                   y = df_g2['mean'],
                   error_y = dict(type = 'data',
                                  array = df_g2['sd']),
                   name = 'Offline')


data = [trace]

layout = go.Layout(title = 'Chapter 3.3 - Scatter & Error Bar (Offline)',
                   xaxis = dict(title = 'Year'),
                   yaxis = dict(title = 'Revenue (mean)'))

fig = go.Figure(data, layout)

fig.show()

#### Bar Plot을 이용해 구현

In [ ]:
df_g1['lower'] = df_g1['mean'] - df_g1['sd']
df_g1['upper'] = df_g1['mean'] + df_g1['sd']

import math

ymax = math.ceil(df_g1['upper'].max() * 1.05)
ymin = math.ceil(df_g1['lower'].max() * 0.95)

In [ ]:
# hover, text 입력 -> 평균값(하한값, 상한값)

df_g1['text'] = df_g1.apply(lambda row: f"{row['mean']/1000:.2f}K ({row['lower']/1000:.2f}K, {row['upper']/1000:.2f}K)", axis=1)
df_g1.head(3)

In [ ]:
channels = list(df_g1['Channel'].unique())

traces = []

for channel in channels:
    dat = df_g1[df_g1['Channel'] == channel]
    traces.append(go.Bar(
        x = dat.year,
        y = dat['mean'],
        error_y = dict(type = 'data',
                       array = dat['sd']),
        text = dat['text'], # hover text 활성화
        name = channel
    ))

data = traces

layout = go.Layout(
    title = 'Chapter 3.3 - Scatter & Error Bar(Offline)',
    xaxis = dict(title = 'Year'),
    yaxis = dict(title = 'Revenue(Mean)',
    range = [0, ymax]))

fig = go.Figure(data, layout)

fig.show()

In [ ]:
# 채널 참조리스트 생성
channels = list(df_g1['Channel'].unique())
# 빈 리스트 생성
traces = []
for channel in channels:
    dat = df_g1[df_g1['Channel'] == channel]
    traces.append(go.Bar(x = dat['year'],
                         y = dat['mean'],
                         error_y = dict(type = 'data',
                                        symmetric = False,  # 비대칭 / True: 대칭(default)
                                        array = dat['sd']
                                       ),
                         text = dat['text'],  # hover text 활성화
                         hoverinfo = 'text',  # 입력한 text만 활성화
                         name = channel
                        ))
data = traces
fig = go.Figure(data)
layout = go.Layout(title = 'Chapter 3.3 - Bar & Error Bar',
                  xaxis = dict(title = 'Year'),
                  yaxis = dict(title = 'Revenue (Mean)', range = [0, ymax]))
fig = go.Figure(data, layout)
fig.show()

### Radar Chart

- 다수의 평가항목이 있는 경우에 사용

In [ ]:
# 연도별 상품 매출액 합계
df_g = df.loc[:,['Category','Revenue','year']].groupby(by = ['year','Category'], as_index=False).sum()
# 매출액 별 순위 생성
df_g['Rank'] = 0
df_g.loc[df_g['Revenue']<10000000, 'Rank'] = 1
df_g.loc[(df_g['Revenue']>=10000000) & (df_g['Revenue']<30000000), 'Rank'] = 2
df_g.loc[(df_g['Revenue']>=30000000) & (df_g['Revenue']<50000000), 'Rank'] = 3
df_g.loc[(df_g['Revenue']>=50000000) & (df_g['Revenue']<70000000), 'Rank'] = 4
df_g.loc[(df_g['Revenue']>=70000000), 'Rank'] = 5
df_g.head()

In [ ]:
d20 = df_g[df_g.year == '2020'].copy()
trace = go.Scatterpolar(
    r = list(d20.Rank),  # 평가 점수
    theta = list(d20.Category),  # 평가 항목
    fill = 'toself',  # 내부 음영
    name = '2020'
)

data = [trace]

layout = go.Layout(title = 'Chapter 3.4 - Radar Chart')

fig = go.Figure(data, layout)

fig.show()

In [ ]:
ranks = list(d20.Rank)
ranks.append(ranks[0])
thetas = list(d20.Category)
thetas.append(thetas[0])

print(ranks, '\n', thetas)

In [ ]:
years = list(df_g.year.unique())  # 연도 리스트 생성
years.sort()                      # 오름차순 정렬
traces = []

for year in years:
    dat = df_g[df_g.year == year]  # 특정 연도 추출
    ranks = list(dat.Rank)         # 매출 순위 리스트
    ranks.append(ranks[0])         # 마지막 연결부 추가
    thetas = list(dat.Category)    # 상품 리스트
    thetas.append(thetas[0])       # 마지막 연결부 추가
    traces.append(go.Scatterpolar(r = ranks,        # 평가 점수
                                  theta = thetas,   # 평가 항목
                                  name = year))


data = traces

layout = go.Layout(
    title = 'Chater 3.4 - Radar Chart',
    legend_orientation = 'h',      # 범주 수평 나열
    legend = dict(                 # 범주 위치 조정
        x = 0.3,
        y = -0.1
    ))

fig = go.Figure(data, layout)

fig.show()

### Indicator

- 기본값 대비 차이값을 지표값으로 표현

|옵션|내용|
|-|-|
|'gauge'|게이지 형식으로 출력 (디폴트값)|
|'number'|주요값만 출력|
|'delta'|차이값만 출력|
|조합|'number + delta', 'gauge+number+delta'등 여러 조합 가능|

In [ ]:
# 이익(Margin) 생성
df['Margin'] = df['Revenue'] - df['Cost']
# 2020년도 매출 및 이익
df_g = df[df['year']=='2020'].copy()
# 수치 출력 조정 (10만 단위)
df_g1 = round(df_g.loc[:,['Revenue','Margin']].sum()/1000000,2)

In [ ]:
trace1 = go.Indicator(value = 200,
                      delta = dict(reference = 160),
                      gauge = dict(axis = dict(visible = False)),
                      domain = dict(row = 0, column = 0))
trace2 = go.Indicator(value = 120,
                      gauge = dict(shape = 'bullet'),
                      domain = dict(x = [0.05, 0.5], y = [0.15, 0.35]))
trace3 = go.Indicator(mode = 'number+delta',
                      value = 300,
                      domain = dict(row = 0, column = 1))
trace4 = go.Indicator(mode = 'delta',
                      value = 40,
                      domain = dict(row = 1, column = 1))
data = [trace1, trace2, trace3, trace4]
layout = go.Layout(grid = {'rows': 2, 'columns': 2, 'pattern' : 'independent'},
                   template = {'data' : {'indicator' :
                                         [{'title' : {'text' : 'Speed'},
                                           'mode' : 'number+delta+gauge',
                                           'delta' : {'reference': 90}}]}})
fig = go.Figure(data, layout)
fig.show()

In [ ]:
values = df_g1['Revenue']
deltas = df_g1['Revenue'] - df_g1['Margin']
trace = go.Indicator(mode = 'number+delta',                           # 출력 방식
                     value = values,                                  # 주요값 입력
                     number = dict(prefix = '$',                      # 주요값 앞 문자열
                                   suffix = 'M',                      # 주요값 뒤 문자열
                                   valueformat = ',0f'),              # 값 형식
                     delta = dict(reference = deltas,                 # 차이값 입력
                                  valueformat = '.2f',                # 값 형식
                                  relative = False,
                                  increasing = dict(color = 'blue'),  # 증가 시 색상
                                  position = 'top'))                  # 차이값 위치
data = [trace]
layout = go.Layout(title = 'Chatper 3.5 - Indicator',
                   paper_bgcolor = "white")                           # 배경 흰색
fig = go.Figure(data, layout)
fig.show()